Imports

In [50]:
import nltk
import pandas as pd
import numpy as np
import re

Wczytanie zbioru uczącego oraz testującego

In [118]:
training_set = pd.read_csv('train.csv')
training_set2 = training_set[["Reviews", "Rating"]].fillna(' ')
reviews = training_set2["Reviews"]
rating = training_set["Rating"]
test_set = pd.read_csv('test.csv')
test_set_rev = test_set["Reviews"]

Przeprowadzenie prostej tokenizacji oraz utworzenie macierzy częstości słów

In [119]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,2),tokenizer = token.tokenize)
text_counts= cv.fit_transform(reviews)

Podział zbioru na treningowy oraz testowy

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, rating, test_size=0.3, random_state=1)

Test wielomianowego naiwnego klasyfikatora Bayesa

In [5]:
'''
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import math
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print(predicted)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))
print("RMSE: ", math.sqrt(mean_squared_error(y_test, predicted)))
'''

[5 5 1 ... 5 5 5]
MultinomialNB Accuracy: 0.7314513210278683
RMSE:  1.0383683344933554


Regresja logistyczna metodą Newtona

In [62]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(warm_start = True, solver = 'newton-cg', multi_class='multinomial', random_state = 23)
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=23, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=True)

Sprawdzenie trafności oraz RMSE dla powyższego modelu

In [63]:
import math
from sklearn import metrics
from sklearn.metrics import mean_squared_error
lr_prediction = logreg.predict(X_test)
print(lr_prediction)
print("Accuracy:",metrics.accuracy_score(y_test, lr_prediction))
print("RMSE: ", math.sqrt(mean_squared_error(y_test, lr_prediction)))

[5 3 1 ... 5 4 5]
Accuracy: 0.8850228671075576
RMSE:  0.6185104636383383


Ponownie regresja logistyczna metodą Newtona. Tym razem na pełnym dostarczonym zbiorze treningowym.

In [120]:
logreg = LogisticRegression(warm_start = False, solver = 'newton-cg', multi_class='multinomial', random_state = 23)
logreg.fit(text_counts, rating)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=23, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

Próba dokonania predykcji przy pomocy powyższego modelu na dostarczonym zbiorze testowym

In [123]:
test_set_counts = cv.transform(test_set_rev.fillna(' '))
sub = logreg.predict(test_set_counts)
df_sub = pd.DataFrame(sub)
print(df_sub)
df_sub.to_csv('my_sub.csv', index=True, header = false)

        0
0       5
1       5
2       5
3       5
4       5
5       5
6       5
7       5
8       5
9       5
10      5
11      5
12      5
13      5
14      5
15      5
16      5
17      5
18      5
19      5
20      5
21      5
22      5
23      5
24      5
25      5
26      5
27      5
28      5
29      5
...    ..
211165  4
211166  5
211167  5
211168  2
211169  5
211170  5
211171  5
211172  1
211173  5
211174  5
211175  1
211176  4
211177  5
211178  5
211179  5
211180  5
211181  5
211182  4
211183  1
211184  3
211185  5
211186  1
211187  5
211188  5
211189  1
211190  5
211191  5
211192  5
211193  1
211194  5

[211195 rows x 1 columns]
